## Creating confidence intervals in python
In this assessment, you will look at data from a study on toddler sleep habits. 

The confidence intervals you create and the questions you answer in this Jupyter notebook will be used to answer questions in the following graded assignment.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import t
pd.set_option('display.max_columns', 30) # set so can see all columns of the DataFrame

Your goal is to analyse data which is the result of a study that examined
differences in a number of sleep variables between napping and non-napping toddlers. Some of these
sleep variables included: Bedtime (lights-off time in decimalized time), Night Sleep Onset Time (in
decimalized time), Wake Time (sleep end time in decimalized time), Night Sleep Duration (interval
between sleep onset and sleep end in minutes), and Total 24-Hour Sleep Duration (in minutes). Note:
[Decimalized time](https://en.wikipedia.org/wiki/Decimal_time) is the representation of the time of day using units which are decimally related.   


The 20 study participants were healthy, normally developing toddlers with no sleep or behavioral
problems. These children were categorized as napping or non-napping based upon parental report of
children’s habitual sleep patterns. Researchers then verified napping status with data from actigraphy (a
non-invasive method of monitoring human rest/activity cycles by wearing of a sensor on the wrist) and
sleep diaries during the 5 days before the study assessments were made.


You are specifically interested in the results for the Bedtime, Night Sleep Duration, and Total 24-
Hour Sleep Duration. 

ref: Akacem LD, Simpkin CT, Carskadon MA, Wright KP Jr, Jenni OG, Achermann P, et al. (2015) The Timing of the Circadian Clock and Sleep Differ between Napping and Non-Napping Toddlers. PLoS ONE 10(4): e0125181. https://doi.org/10.1371/journal.pone.0125181

In [2]:
# Import the data (use this if running your Jupyter notebook within Coursera)
df = pd.read_csv("nap_no_nap.csv") 

In [0]:
# Imort the data (uncomment the line below and use this if you downloaded the Jupyter notebook)
# df = pd.read_csv("https://raw.githubusercontent.com/UMstatspy/UMStatsPy/master/Course_2/nap_no_nap.csv")

In [5]:
# First, look at the DataFrame to get a sense of the data
df.head(10)

,id,sex,age (months),dlmo time,days napped,napping,nap lights outl time,nap sleep onset,nap midsleep,nap sleep offset,nap wake time,nap duration,nap time in bed,night bedtime,night sleep onset,sleep onset latency,night midsleep time,night wake time,night sleep duration,night time in bed,24 h sleep duration,bedtime phase difference,sleep onset phase difference,midsleep phase difference,wake time phase difference
0,1,female,33.7,19.24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.45,20.68,0.23,1.92,7.17,629.4,643.00,629.40,-1.21,-1.44,6.68,11.93
1,2,female,31.5,18.27,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.23,19.48,0.25,1.09,6.69,672.4,700.40,672.40,-0.96,-1.21,6.82,12.42
2,3,male,31.9,19.14,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.60,20.05,0.45,1.29,6.53,628.8,682.60,628.80,-0.46,-0.91,6.15,11.39
3,4,female,31.6,19.69,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.46,19.50,0.05,1.89,8.28,766.6,784.00,766.60,0.23,0.19,6.20,12.59
4,5,female,33.0,19.52,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.21,19.65,0.45,1.30,6.95,678.0,718.00,678.00,0.31,-0.13,5.78,11.43
5,6,female,36.2,18.22,4,1,14.00,14.22,15.00,15.78,16.28,93.75,137.00,19.95,20.25,0.29,1.26,6.28,602.2,653.80,695.95,-1.73,-2.03,7.05,12.06
6,7,male,36.3,19.28,1,1,14.75,15.03,15.92,16.80,16.08,106.00,80.00,20.60,20.96,0.36,2.12,7.27,618.4,655.40,724.40,-1.32,-1.68,6.84,11.99
7,8,male,30.0,21.06,5,1,13.09,13.43,14.44,15.46,15.82,121.60,163.80,22.01,22.53,0.51,2.92,7.31,526.8,582.40,648.40,-0.95,-1.47,5.86,10.25
8,9,male,33.2,19.38,2,1,14.41,14.42,15.71,17.01,16.60,155.50,131.25,20.24,20.37,0.13,1.60,6.82,626.8,660.33,782.30,-0.86,-0.99,6.22,11.44
9,10,female,37.1,19.93,3,1,13.12,13.42,14.31,15.19,15.30,106.67,130.67,20.78,21.63,0.84,2.20,6.52,549.5,626.00,656.17,-0.76,-1.82,6.21,10.59


In [6]:
# Number of rows in the dataframe
len(df)

20

In [7]:
df['napping'].value_counts()

1    15
0     5
Name: napping, dtype: int64

**Question**: What variable is used in the column 'napping' to indicate a toddler takes a nap?  
1

**Question**: What is the sample size $n$? What is the sample size for toddlers who nap, $n_1$, and toddlers who don't nap, $n_2$?<br>
$n$ = 20<br>
$n_1$ = 15<br>
$n_2$ = 5

### Average bedtime confidence interval for napping and non napping toddlers
Create two 95% confidence intervals for the average bedtime, one for toddler who nap and one for toddlers who don't.

Before any analysis, we will convert 'night bedtime' into decimalized time.

In [8]:
# Convert 'night bedtime' into decimalized time
df.loc[:,'night bedtime'] = np.floor(df['night bedtime'])*60 + np.round(df['night bedtime']%1,2 )*100

Now, isolate the column 'night bedtime' for those who nap into a new variable, and those who didn't nap into another new variable. 

In [13]:
bedtime_nap = df.loc[df.loc[:,'napping']==1,'night bedtime']

In [14]:
bedtime_no_nap = df.loc[df.loc[:,'napping']==0,'night bedtime']

Now find the sample mean bedtime for nap and no_nap.

In [15]:
nap_mean_bedtime = bedtime_nap.mean()
nap_mean_bedtime

1233.0666666666666

In [16]:
no_nap_mean_bedtime = bedtime_no_nap.mean()
no_nap_mean_bedtime

1191.0

Now find the standard error for $\bar{X}_{nap}$ and $\bar{X}_{no\ nap}$.

In [19]:
nap_se_mean_bedtime = bedtime_nap.std()/np.sqrt(15)
nap_se_mean_bedtime

8.893800230479801

In [20]:
no_nap_se_mean_bedtime = bedtime_no_nap.std()/np.sqrt(5)
no_nap_se_mean_bedtime

15.339491516996253

**Question**: Given our sample sizes of $n_1$ and $n_2$ for napping and non napping toddlers respectively, how many degrees of freedom ($df$) are there for the associated $t$ distributions?

(14, 4)

To build a 95% confidence interval, what is the value of t\*?  You can find this value using the percent point function: 
```
from scipy.stats import t

t.ppf(probabiliy, df)
```
This will return the quantile value such that to the left of this value, the tail probabiliy is equal to the input probabiliy (for the specified degrees of freedom). 

Example: to find the $t^*$ for a 90% confidence interval, we want $t^*$ such that 90% of the density of the $t$ distribution lies between $-t^*$ and $t^*$. Or in other words if $X \sim t(df)$:  
$P(-t^*< X < t^*) = .90$  
Which, because the $t$ distribution is symmetric, is equivalent to finding $t^*$ such that:  
$P(X < t^*) = .95$

So the $t^*$ for a 90% confidence interval, and lets say df=10, will be:
```
t_star = t.ppf(.95, df=10)
```

In [42]:
# Find the t_stars for the 95% confidence intervals
nap_t_star = t.ppf(0.975, df=14)
nap_t_star

2.1447866879169273

In [43]:
no_nap_t_star = t.ppf(0.975, df=4)
no_nap_t_star

2.7764451051977987

**Quesion**: What is $t^*$ for nap and no nap?

$t^*$ for nap = 2.1448<br>
$t^*$ for no nap = 2.7764<br>

Now to create our confidence intervals. For the average bedtime for nap and no nap, find the upper and lower bounds for the respective confidence intervals.

In [47]:
# nap
ucb = nap_mean_bedtime + nap_t_star*nap_se_mean_bedtime
lcb = nap_mean_bedtime - nap_t_star*nap_se_mean_bedtime
(lcb, ucb)

(1213.991362327341, 1252.1419710059922)

In [46]:
# no_nap
ucb_1 = no_nap_mean_bedtime + no_nap_t_star*no_nap_se_mean_bedtime
lcb_1 = no_nap_mean_bedtime - no_nap_t_star*no_nap_se_mean_bedtime
(lcb_1, ucb_1)

(1148.4107438614126, 1233.5892561385874)

**Question**: What are the 95% confidence intervals, rounded to the nearest ten, for the average bedtime (in decimalized time) for toddlers who nap and for toddlers who don't nap? 

CI = $\bar{X} \pm \ t^* \cdot s.e.(\bar{X})$

**Challenge problem 1**: Write a function that inputs the column containing the data you want to build your confidence interval from and returns the confidence interval as a list or tuple (i.e. \[upper, lowe\] or (upper, lower)).

**Challenge problem 2**: Create the intervals using the statsmodels function stats.weightstats.DescrStatsW:

```
stats.weightstats.DescrStatsW(data).tconfint_mean(alpha=.05)
```

In [0]:
from statsmodels import stats

